In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
import zipfile

In [ ]:
zip_file_path = r'/content/drive/MyDrive/Uplift Datasets/Customer Support Ticket Dataset.zip'
file_folder_path = r'/content/drive/MyDrive/Uplift Datasets/'

In [ ]:
with zipfile.ZipFile(zip_file_path) as zip_ref:
  zip_ref.extractall(file_folder_path)

In [ ]:
file_path = r'/content/drive/MyDrive/Uplift Datasets/customer_support_tickets.csv'

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
duplicate = df.duplicated().any()
duplicate

So there are no duplicate rows in the data

In [ ]:
cat_cols  = df.select_dtypes(['object']).columns
int_cols  = df.select_dtypes(['int64']).columns
float_cols = df.select_dtypes(['float']).columns
print(cat_cols)
print(int_cols)
print(float_cols)

# Cleaning the Data

In [ ]:
#Convert to datetime
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], format = '%Y-%m-%d')
df['First Response Time'] = pd.to_datetime(df['First Response Time'], format = '%Y-%m-%d %H:%M:%S')
df['Time to Resolution'] = pd.to_datetime(df['Time to Resolution'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
df_ticket = df.drop(columns = ['Ticket ID'], axis = 1)

In [ ]:
#Cleaning data
df_ticket['Resolution'] = df_ticket['Resolution'].fillna('None')
df_ticket['First Response Time'] = df_ticket['First Response Time'].fillna('No response')
df_ticket['Time to Resolution'] = df_ticket['Time to Resolution'].fillna('No resolution')
df_ticket['Customer Satisfaction Rating'] = df_ticket['Customer Satisfaction Rating'].fillna('No rating')

In [ ]:
df_ticket.isnull().sum()

This was done to check if there is anything missing after the data cleaning

# Exploratory Part of the Whole Dataset

### Distribution of Age

In [ ]:
print(df_ticket['Customer Age'].max())
print(df_ticket['Customer Age'].min())

💬 Comment:

The range of customer age from 18 to 70 years old.
I will class them into 3 kinds of customer: Young, Middle Age and Old.

In [ ]:
age = []
for result in df_ticket['Customer Age']:
    if result<=30:
        age.append('Young Customer')
    elif 30<result<55:
        age.append('Middle Age Customer')
    else:
        age.append('Old Customer')
df_ticket['Type of Customer'] = age

In [ ]:
chart_age = df_ticket['Type of Customer'].value_counts()
chart_gen = df_ticket['Customer Gender'].value_counts()
plt.figure(figsize = (10,5))
plt.pie(chart_age, labels = chart_age.index, autopct='%.0f%%')
plt.title('Distribution of customer age', loc = 'left', pad = 10, size = 15)
plt.show()

💬 Comment: Most of customer age is more than 30 and the middle age customer is the main target buying most produce.

### Distribution of Gender

In [ ]:
chart_gen = df_ticket['Customer Gender'].value_counts()
plt.figure(figsize = (10,5))
plt.pie(chart_gen, labels = chart_gen.index, autopct='%.0f%%')
plt.title('Distribution of customer gender', loc = 'left', pad = 10, size = 15)
plt.show()

💬 Comment: Customer in this dataset reach a balance value in distribution of gender.

### Distribution of Product

In [ ]:
unique = len(df_ticket['Product Purchased'].unique())
print(f' Total of products was purchased: {unique}')

In [ ]:
#Plot chart
plt.figure(figsize=(15, 10))

# Count and sort products by frequency
product_counts = df_ticket['Product Purchased'].value_counts(ascending=False).head(10)

# Use seaborn.barplot to plot the distribution with custom order
sns.barplot(x=product_counts.values, y=product_counts.index, palette='viridis')

plt.title('Distribution of Products (Highest to Lowest Sales)', loc='center', pad=10, size=15)
plt.xlabel('Number of Sales')
plt.ylabel('Product Purchased')
plt.show()

💬 Comment:: Cannon EOS, Go Pro Hero, and Nest Thermostat are the top three products have the most pruchased.

### Distribution of Ticket Channel

In [ ]:
chart_chan = df_ticket['Ticket Channel'].value_counts()
plt.pie(chart_chan, labels = chart_chan.index, autopct = '%.2f')
plt.title('Distribution of ticket channel', loc = 'center', pad = 10, size = 15)
plt.show()

💬 Comment: This company has 4 channel for customer to find the technical staff and the usage level of each channel don't have too much different.

### Distribution of Ticket Priority

In [ ]:
chart_prio = df_ticket['Ticket Priority'].value_counts()
plt.pie(chart_prio, labels = chart_prio.index, autopct = '%.2f')
plt.title('Distribution of ticket priority', loc = 'center', pad = 10, size = 15)
plt.show()

💬 Comment: Just like the ticket channel, the cases are ranked into four priority level of ticket and the number of cases in each level are quite similar.

### Chance of Response

In [ ]:
chance_response = []
for result in df_ticket['First Response Time']:
    if result == 'No response':
        chance_response.append('No')
    else:
        chance_response.append('Yes')
df_ticket['Response'] = chance_response

In [ ]:
chart_response = df_ticket['Response'].value_counts()
plt.pie(chart_response, labels = chart_response.index, autopct = '%.2f')
plt.title('Chance of response', loc = 'center', pad = 10, size = 15)
plt.show()

💬 Comment: 66.71% ticket sent was responsed by the technical staff.

### Chance of Resolution

In [ ]:
chance_reso = []
for result in df_ticket['Time to Resolution']:
    if result == 'No resolution':
        chance_reso.append('No')
    else:
        chance_reso.append('Yes')
df_ticket['Resolution_bin'] = chance_reso

In [ ]:
chart_reso = df_ticket['Resolution_bin'].value_counts()
plt.pie(chart_reso, labels = chart_reso.index, autopct = '%.2f')
plt.title('Chance of resolution', loc = 'center', pad = 10, size = 15)
plt.show()

💬 Comment: 32.70% cases sent was resolved by the technical staff.

### Distribution of Customer Satisfaction Rating

💬 Comment: In the data structure, customer satisfaction ratings range from 1 to 5. A higher rating indicates better quality of service provided by the Support Ticket Department.

In [ ]:
#Convert 'Customer Satisfaction Rating' columns into object type
df_ticket['Customer Satisfaction Rating'] = df_ticket['Customer Satisfaction Rating'].astype('object')
df_ticket['Customer Satisfaction Rating'].info()

In [ ]:
chart_satisfaction = df_ticket['Customer Satisfaction Rating'].value_counts()
plt.pie(chart_satisfaction, labels = chart_satisfaction.index, autopct = '%.2f')
plt.title('Distribution of customer satisfaction', loc = 'center', pad = 10, size = 15)
plt.show()

Conclusion:

- The company primarily provides IT products to customers aged 30 and above, serving all genders.

- There is a balanced distribution of cases submitted through the ticketing system across different priority levels.

- Only 32.70% of submitted cases were resolved, indicating a need to improve support efficiency.

- Customer satisfaction ratings were evenly distributed across all levels, showing that the Support Ticket Department performs moderately.

- To enhance performance, the department should focus on increasing resolution rates and improving customer satisfaction through better service quality and responsiveness.

# Discovering Relationships between Columns

### Ticket Channel and Ticket Priority

In [ ]:
plt.figure(figsize = (15,6))
sns.histplot(df_ticket, hue = 'Ticket Channel', x = 'Ticket Priority',multiple = 'dodge', shrink = 0.8)
plt.title('Distribution of Ticket Channel by Priority', loc = 'center', pad = 10, size = 20)
plt.show()

💬 Comment: Although each ticket channel has similar in number of cases was sent, but:

- Email is the channel has the most cases was ranked critical level
- Social media is the channel customers

### Ticket Priority and Solution

In [ ]:
#Create a pivot table for caculating
priority = pd.pivot_table(df_ticket, index = ['Ticket Priority'], values = ['Customer Name'], columns = ['Resolution_bin'], aggfunc = len).reset_index()
priority = priority.rename(columns = {'Customer Name': 'Chance of Solution'})
priority['Percent'] = (priority['Chance of Solution']['Yes']/priority['Chance of Solution']['No'])*100
priority

In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(priority, x = 'Ticket Priority', y = 'Percent', hue = 'Ticket Priority')
plt.title('Chance of solution per ticket priority', loc = 'center', pad = 10, size = 15)
plt.show()

💬 Comment: High and Critical priority ticket only have a slightly higher in chance of solution than low and medium.

### Customer Satisfaction Rating

In [ ]:
#Create a pivot table for caculating
rating = pd.pivot_table(df_ticket, index = ['Customer Satisfaction Rating'], values= ['Customer Name'], columns = ['Resolution_bin'], aggfunc = len).reset_index()
rating = rating.rename(columns = {'Customer Name': 'Resolution'})
rating

💬 Comment: Because all rating from customers were in resolution problems, so I will create a new dataframe contains all data of resolution cases for a deeper look in rating.

In [ ]:
#Create new dataframe
df_resolution = df_ticket[df_ticket['Resolution_bin'] =='Yes']
df_resolution.head(5)

In [ ]:
chart_reprio = df_resolution['Ticket Priority'].value_counts()
plt.pie(chart_reprio, labels = chart_reprio.index, autopct = '%.2f')
plt.title('Distribution of ticket priority in case resolution', loc = 'center', pad = 10, size = 15)
plt.show()

💬 Comment: For a clearly look, I will put "Low" for the score 1 and 2, "Medium" for 3, and "High" for 4 and 5.

In [ ]:
rating = []
for result in df_resolution['Customer Satisfaction Rating']:
    if result<=2:
        rating.append('Low')
    elif result==3:
        rating.append('Medium')
    else:
        rating.append('High')
df_resolution['Type of rating'] = rating

In [ ]:
#Customer satisfaction rating and ticket priority
fig, axes = plt.subplots(1,2, figsize = (20,6))
sns.histplot(df_resolution, x = 'Ticket Priority', hue = 'Type of rating' ,multiple = 'dodge', shrink = 0.5, ax = axes[0])
sns.histplot(df_resolution['Ticket Priority'], shrink = 0.5, ax = axes[1])
axes[0].set_title('Customer satisfaction rating per ticket priority', loc = 'center', pad = 10, size = 15)
axes[1].set_title('Distribution of ticket priority in case resolution', loc = 'center', pad = 10, size = 15)
plt.show()

💬 Comment: As we see above, the distribution of rating scores and priority are quite similar. However:

- Critical tickets are graded highest priority but also a priority have most number of customer gave low rating score
- This priority also reach the highest in number of customers judged high score but the number of cases given low rating were more than high rating.
- The second rank in both low and high rating were high priority.

In [ ]:
#Customer satisfaction rating and ticket type
fig, axes = plt.subplots(1,2, figsize = (15,6))
sns.histplot(df_resolution['Ticket Type'], shrink = 0.8, ax = axes[0])
sns.histplot(hue = df_resolution['Type of rating'], x = df_resolution['Ticket Type'], multiple = 'dodge', shrink = 0.8, stat = 'percent', ax = axes[1])
axes[0].tick_params(labelrotation = 30)
axes[1].tick_params(labelrotation = 30)
axes[0].set_title('Distribution of ticket type', loc = 'center', pad = 10, size = 15)
axes[1].set_title('Customer satisfaction rating per ticket type', loc = 'center', pad = 10, size = 15)
plt.show()

💬 Comment:

- Refund request and technical issue are two problems had more low rating score than high rating score.
- Refund request was the type had the most ticket resolved but also the type got the most low rating score.
- The remain types worked well and kept the amount of high rating score more than low rating score.

# Sentiment Analysis

In [ ]:
# Initialize the text analysis pipeline
sentiment_model = "cardiffnlp/twitter-roberta-base-sentiment"
classifier = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentiment_model)

In [ ]:
sample_text = "I hate you."
classifier(sample_text)

In [ ]:
from sklearn.model_selection import train_test_split

# Get 20% stratified sample by "Customer Gender"
df_smaller, _ = train_test_split(df_ticket, test_size=0.80, stratify=df_ticket['Customer Gender'], random_state=42)
# Keep 80% out, so you get 20% in df_smaller

print(f'Original size: {len(df_ticket)}')
print(f'Sampled size: {len(df_smaller)}')

In [ ]:
# Drop these columns because it is not necessary on the modelling I am formulating

df_smaller.drop(columns=['Customer Email'], inplace = True)
df_smaller.drop(columns=['Customer Name'], inplace = True)

In [ ]:
# Use the classifier model to the dataframe
results = classifier(df_smaller['Ticket Description'].tolist())

#Creating new column in the dataframe to store the results
df_smaller['sentiment_label'] = [result['label'] for result in results]
df_smaller['sentiment_score'] = [result['score'] for result in results]

In [ ]:
df_smaller.head(5)

# Preprocessing

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Drop the problematic columns from df_ticket to create a processed DataFrame
# Explicitly drop 'Customer Satisfaction Rating' as it has mixed types after fillna (No Rating)
df_smaller_processed = df_ticket.drop(columns=['Date of Purchase', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating'])

# Define the numerical columns from the processed DataFrame
num_cols = df_smaller_processed.select_dtypes(include=['int', 'float']).columns.tolist()

# Define the categorical columns from the processed DataFrame
# Now, select all object columns since 'Customer Satisfaction Rating' is removed
cat_cols = df_smaller_processed.select_dtypes(include='object').columns.tolist()


trans_pipeline = ColumnTransformer([
    ("num", SimpleImputer(strategy='median'), num_cols),
    ("scaler", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

In [ ]:
df_smaller_trans = trans_pipeline.fit_transform(df_smaller_processed)

# Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA

pca_model = PCA(n_components=2)
df_pca = pca_model.fit_transform(df_smaller_trans)

In [ ]:
plt.figure(figsize = (5,7))
plt.scatter(x = df_pca[:, 0], y = df_pca[:, 1], cmap='viridis', s= 50)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Principal Components')
plt.show()

# KMeans Clustering

In [ ]:
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans_model.fit_predict(df_pca)

In [ ]:
plt.figure(figsize = (5,7))
plt.scatter(x = df_pca[:, 0], y = df_pca[:, 1], c=clusters ,cmap='viridis', s= 50)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Principal Components')
plt.show()

# Elbow Method

In [ ]:
inertias = []
number_clusters = range(1, 15)

for number_cluster in number_clusters:
  kmeans_model = KMeans(n_clusters = number_cluster, random_state=42, n_init=10)
  kmeans_model.fit(df_pca)
  inertias.append(kmeans_model.inertia_)

In [ ]:
plt.figure(figsize=(10,4))
sns.scatterplot(x=number_clusters, y=inertias)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia Score')
plt.title('Number of Cluster vs Inertia')
plt.show()

In [ ]:
df_smaller_processed['Cluster'] = clusters

In [ ]:
df_smaller_processed.head(5)

# Zero Shot Classification

In [ ]:
from transformers import pipeline

candidate_labels = ['Refund Request', 'Complaint', 'Technical Problem', 'Feedback', 'Suggestion', 'Product Inquiry']

In [ ]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

In [ ]:
sample_text = 'Im facing a problem with my product purchased.'

In [ ]:
result = classifier(sample_text, candidate_labels)

In [ ]:
result['labels'][0]

In [ ]:
from sklearn.model_selection import train_test_split

# Get a smaller sample from df_smaller for zero-shot classification
# Use a simple random split instead of stratifying by 'Ticket Description'
df_zeroshot, _ = train_test_split(df_smaller, test_size=0.75, random_state=42)


print(f"Original size (df_smaller): {len(df_smaller)}")
print(f"Sampled size (df_zeroshot): {len(df_zeroshot)}")

In [ ]:
# Function to classify topics with token truncation
def get_topic(text):
    if not isinstance(text, str):
        return None

    result = classifier(
        text,
        candidate_labels,
        max_length=200,       # Limit to 200 tokens
        truncation=True       # Automatically truncate longer text
    )

    # Return only the most probable topic
    return result['labels'][0]

In [ ]:
df_zeroshot['Topic'] = df_zeroshot['Ticket Description'].apply(get_topic)

In [ ]:
df_zeroshot['Topic'].value_counts()

# Unsupervised Topic Modelling

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(stop_words = "english", max_features = 10)

In [ ]:
vectorized_text = vectorizer.fit_transform(df['Ticket Description'])

In [ ]:
def get_keywords(row_index, top_n_words = 5):
  row = vectorized_text[row_index].toarray()[0]
  top_indices = row.argsort()[-top_n_words:][::-1] # indetify the index of the top keywords

  return [vectorizer.get_feature_names_out()[i] for i in top_indices]

In [ ]:
df_ticket['keywords'] = [get_keywords(i) for i in range(len(df_ticket))]

In [ ]:
df['keywords']

In [ ]:
from wordcloud import WordCloud

def plot_wordcloud(text, mask=None, color='white', max_font_size=None, title_size=None, title="WordCloud"):

    wordcloud = WordCloud(width=800, height=800,
                          background_color=color,
                          mask=mask,
                          stopwords=None,
                          max_font_size=max_font_size,
                          min_font_size=10).generate(text)

    plt.figure(figsize=(8, 8), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)

    plt.title(title, size=title_size)
    plt.show()

# Flatten the list of keywords
all_keywords = [keyword for sublist in df['keywords'] for keyword in sublist]

# Join the keywords into a single string
text = " ".join(all_keywords)

# Generate the word cloud using the defined function
plot_wordcloud(text, title="WordCloud of Keywords")

In [ ]:
# Flatten the list of keywords for resolved tickets
resolved_keywords = [keyword for sublist in df_ticket[df_ticket['Resolution_bin'] == 'Yes']['keywords'] for keyword in sublist]

# Join the keywords into a single string
resolved_text = " ".join(resolved_keywords)

plot_wordcloud(resolved_text, mask=None, color='white', max_font_size=50, title_size=30, title="WordCloud of Tickets resolved")

In [ ]:
# Flatten the list of keywords for unresolved tickets
resolved_keywords = [keyword for sublist in df_ticket[df_ticket['Resolution_bin'] == 'No']['keywords'] for keyword in sublist]

# Join the keywords into a single string
resolved_text = " ".join(resolved_keywords)

plot_wordcloud(resolved_text, mask=None, color='white', max_font_size=50, title_size=30, title="WordCloud of Tickets Unresolved")

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

def make_wordcloud_for_cluster(cluster_id, clusters, vectorized_text, vectorizer, max_words=100):
    """
    cluster_id: integer label of the cluster
    clusters: array or list of cluster labels for each document
    vectorized_text: TFIDF matrix (n_docs x n_terms)
    vectorizer: the fitted TfidfVectorizer

    Returns a WordCloud object (and also plots it).
    """
    # find documents in that cluster
    doc_indices = np.where(clusters == cluster_id)[0]
    if len(doc_indices) == 0:
        print(f"No documents in cluster {cluster_id}")
        return None

    # sum the TF-IDF vectors of those docs
    # this gives a vector of size n_terms with aggregate “importance” in this cluster
    cluster_tfidf_sum = vectorized_text[doc_indices].sum(axis=0)  # this is a 1×n_terms sparse or dense row
    # convert to array
    if hasattr(cluster_tfidf_sum, "A1"):  # for scipy sparse result
        freq = cluster_tfidf_sum.A1
    else:
        # if already dense
        freq = cluster_tfidf_sum.flatten()

    # get mapping term -> weight
    terms = vectorizer.get_feature_names_out()
    term_weights = {terms[i]: freq[i] for i in range(len(terms)) if freq[i] > 0}

    # generate word cloud
    wc = WordCloud(width=800, height=400, background_color="white")
    wc = wc.generate_from_frequencies(term_weights)

    # plot
    plt.figure(figsize=(10, 5))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Cluster {cluster_id} Word Cloud")
    plt.show()

    return wc

# Suppose your cluster labels are in `clusters`
# And you know how many clusters (say k)
k = len(set(clusters))  # or your chosen k

for cid in range(k):
    make_wordcloud_for_cluster(cid, clusters, vectorized_text, vectorizer, max_words=100)